In [131]:
!nvidia-smi

Fri Dec  2 04:55:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    34W /  70W |   4272MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [132]:
!pip install -q transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# tokenizer
tokenizer_checkpoint = "Salesforce/codegen-350M-multi"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

# model finetuned on CSS
checkpoint = "alecsharpie/codegen_350m_css"
css_model = AutoModelForCausalLM.from_pretrained(checkpoint)

# model finetuned on html
checkpoint = "alecsharpie/codegen_350m_html"
html_model = AutoModelForCausalLM.from_pretrained(checkpoint)


# move model to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

css_model.to(device)
html_model.to(device)


#### HTML

In [134]:
%%time

page_title = "Thank you"


html_boilerplate = f"""
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8" />
        <meta name="viewport" content="width=device-width, initial-scale=1.0" />
        <title>{page_title}</title>
    </head>
    <body>"""

batch = tokenizer(html_boilerplate, return_tensors="pt")

batch = {k: v.to(device) for k, v in batch.items()}

completion = html_model.generate(**batch, max_new_tokens = 512)

html_output = tokenizer.decode(completion[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 11.3 s, sys: 30 ms, total: 11.3 s
Wall time: 12.9 s


In [135]:
# prepare html to use as input to css model
html_output = html_output.replace('<|endoftext|>', '')

html_output = html_output.split('</body>')[0] + "</body>\n<style>"

#### CSS

In [136]:
%%time

# generate css code
batch = tokenizer(html_output, return_tensors="pt")

batch = {k: v.to(device) for k, v in batch.items()}

completion = css_model.generate(**batch, max_new_tokens = 512)

code_output = tokenizer.decode(completion[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 17.6 s, sys: 59.5 ms, total: 17.7 s
Wall time: 18.6 s


In [137]:
# parse generated code into full format

code_output = code_output.replace('<|endoftext|>', '')

if code_output[-1] != '}':
  code_output  = code_output + '}'

code_output = code_output + "</style>\n</html>"


#### Results

In [138]:
from IPython.display import display, HTML
display(HTML(code_output))

#### Download file

In [139]:
from google.colab import files

# save code to file and download
with open('index.html','w') as f:
  f.write(code_output)

files.download("index.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>